In [8]:
import torch
import torchvision.models.video as models
import torch.optim as optim
from tqdm import tqdm
from FitnessData import FitnessData
from torch.utils.data import DataLoader, random_split

In [3]:
model = models.r2plus1d_18(pretrained=True)

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R2Plus1D_18_Weights.KINETICS400_V1`. You can also use `weights=R2Plus1D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/r2plus1d_18-91a641e6.pth" to /Users/zhumingyu/.cache/torch/hub/checkpoints/r2plus1d_18-91a641e6.pth


  0%|          | 0.00/120M [00:00<?, ?B/s]

In [4]:
model.fc = torch.nn.Linear(in_features=model.fc.in_features, out_features=22)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()  # Standard loss for classification
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [ ]:
def train(model, dataloader, criterion, optimizer, num_epoch=10):
    model.train()
    for i in range(num_epoch):
        running_loss = 0.0
        correct = 0
        total = 0
        
        loop = tqdm(dataloader, leave=True)
        for video, label in loop:
            video, label = video.to(device), label.to(device)
            optimizer.zero_grad()
            output = model(video)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = output.max(1)
            correct += (predicted == label).sum().item()
            total += label.size(0)

            loop.set_description(f"Epoch [{i+1}/{num_epoch}]")
            loop.set_postfix(loss=running_loss / total, acc=100. * correct / total)

    print(f"Finished Training. Final Accuracy: {100 * correct / total:.2f}%")

In [19]:
dataset = FitnessData(root_dir="workoutfitness-video", transform=None, frames_per_clip=16)
train_size= int(0.7*(len(dataset)))
val_size = int(0.15*(len(dataset)))
test_size= len(dataset) - train_size - val_size

In [20]:
train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

In [22]:
print(f"Train_size: {train_size}, Val_size: {val_size}, Test_size: {test_size}")

Train_size: 414, Val_size: 88, Test_size: 90


In [24]:
# Define batch size
batch_size = 4

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)


In [25]:
def validate(model, val_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for videos, labels in val_loader:
            videos, labels = videos.to(device), labels.to(device)
            outputs = model(videos)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    return 100 * correct / total  # Return accuracy percentage

In [ ]:
train(model=model, dataloader=train_loader, criterion=criterion, optimizer=optimizer, num_epoch=10)